# First steps with COSIPY

This is the first tutorial to get you started with using COSIPY. COSIPY can be run in two ways: Either by running `COSIPY.py` from the command line

`
$ python COSIPY.py
`

This will read the model configuration from the files `constants.py` and `config.py`, run the model and save the results to a netcdf file on disk. In this case any customization to the run is done by changing the relevant variable(s) in the previously mentioned files. It is also possible to run COSIPY in an interactive session, or from your own scripts. This is what we are going to go through in these tutorials. In this notebook you'll learn how to:

* Setup a simulation for a single gridpoint
* Run the simulation and save the results
* Do simple visualisations of the results using matplotlib

First we have to import the python libraries we are going to use

In [ ]:
# Have to change the cwd for the ipython session, otherwise COSIPY
# will look for things in the wrong places.
import os
import sys
# This is not really a good method, if cell is re run we end up in the
# wrong directory.
os.chdir('./../')
sys.path.append(os.getcwd())

In [ ]:
# edu_utils has some quality of life functions
from cosipy.utils import edu_utils
import numpy as np
from matplotlib import pyplot as plt

In [ ]:
# Have to tell matplotlib to plot inline
%matplotlib inline

# Create input/output files

First we have to create the input and output datasets for the simulation. This is handled by the `IOClass` of COSIPY, which is not only responsible for creating the [xarray](http://xarray.pydata.org/en/stable/index.html) datasets holding the input and output data, but also for updating these datasets with new data after a run and saving the output to a netcdf file.

Before we initialize the `IO`, it is good to take a look at the variables and constants used by the `IO` module, and also the rest of the model. We can plot the defaults of the most interesting ones using the `print_options` from the `edu_utils`.

In [ ]:
edu_utils.print_options()

These are loaded from the files `config.py` and `constants.py` located in the root directory of cosipy. We can change any of these variables to customize our simulation. This can by done changing the value of a variable in the previously mentioned files, however for this to take affect we have to reload the `edu_utils` module.  

A much simpler way is to store the variable we want to change in in the `opt_dict`. This is a dictionary which is passed along to the different modules of the model and overwrites the default variable. We can specify new values for any number of variables available in the list above. For instance we can change the variable `time_end` to adjust the temporal extent of our simulation:

In [ ]:
# Inititalize an empty dictionary.
opt_dict = dict()
# Create a key-value pair for time end.
opt_dict['time_end'] = '2009-01-08T00:00'

We are now ready to initialize the `IOClass` and the datasets holding the input and output data for our simulation. For this we have a utility function `create_IO`, which takes the `opt_dict` and returns an instance of the `IOClass` and the `DATA` and `RESULTS` datasets.

In [ ]:
IO, DATA, RESULTS = edu_utils.create_IO(opt_dict)

The output above gives us some helpful insight about the data. We can see that the data covers the period 2009-01-01 to 2009-01-31, and that we have specified the integration time to 2009-01-01 to 2009-01-08 by adding a new value to `time_end` in the `opt_dict`. We also see that we have one gridpoint which we will perform the calculations on.

It also shows some quality control on the input data, more on this later.

<div class="alert alert-success">
    <b>Note</b> that we don't have to use the <b>opt_dict</b> as we did above - if we don't, the simulation will run using the default parameters from config.py and constants.py.

Also note that the IOClass along with the DATA and RESULTS datasets has to be <b>re-initliazed</b> after a variable has been changed/added to the <b>opt_dict</b>.
</div>

# Running COSIPY
Now we are ready to do a simple point simulation with COSIPY. This example is run on the Zhadang glacier. We can change this by changing the `input_netcdf` via the `opt_dict` to point at the files we want to use. In these tutorials data for Zhadang and Hintereisferner are included.


We can then run the model by simply calling the `run_model` function from the `edu_utils`

<div class="alert alert-success">
    <b>Note that this can take some time.</b>
</div>

In [ ]:
edu_utils.run_model(DATA, IO, RESULTS, opt_dict)

Thats it! Now we are ready to inspect the results.

# Visualizing the results
To get a quick overview of the variables in our dataframe, we can simply call it 

In [ ]:
RESULTS

We can see that our data is stored in an xarray dataset - a very handy format for interacting with netcdf files in python. To check which data variables are available, one can simply click on the label "Data Variables", which will expand a list of the variables.

We can quckly plot a variable, the surface mass balance (`surfMB`) for instance:

In [ ]:
RESULTS.surfMB.plot();

As you can see, the use of xarray-datasets for storing the data makes it possible to levarage the built in plotting functionality of xarray and quickly visualize the output.

For a slightly more advanced plot, one can still make use of matplotlib

In [ ]:
fig, (ax1, ax2) = plt.subplots(figsize=(7,8), nrows=2, sharex=True,
                               gridspec_kw = {'height_ratios':[2,1]})
RESULTS.surfMB.plot(ax=ax1)
RESULTS.TS.plot(ax=ax2, color='C1')
# Easier to make use of xarray plotting capabilites and then remove unwanted lables.
ax1.set_xlabel('')
ax2.set_title('')
# Get the time span of our dataset to use for title
time_span = RESULTS.time[0].values, RESULTS.time[-1].values
time_span = np.datetime_as_string(time_span, unit='m')
# Set the title of the plot.
ax1.set_title(f'Surface mass balance and Snowfall at Zhadang \n between {time_span[0]} and' +\
              f' {time_span[1]}')
# Fixes the spacing between parts of the plots.
fig.tight_layout()

**Notice the scale of the surface mass balance.** In the plot above, we see the surface mass blance (blue) and the 2-meter air temperature (orange). Even though the air temperautre stays below 0°C (273.15K) for the entire period, the mass balance is still negative during the day. We will investigate this is the following section.

<div class="alert alert-warning">
    <details>
    <Summary><b>Question: Can you think of a variable to plot as a way to explain the positive surface mass balance during the 6th of January?</b> <i>Click me for a hint</i></Summary>
        Try plotting the snowfall, i.e. RESULTS.SNOWFALL.plot().
    </details>
</div>

In [ ]:
# Write your code here


## A look at what is causing the melt

Since the temperature never reaches above zero, melt is not causing the small loss of mass during the days seen in the plot above. Instead it is sublimation, the process when a substance transitions from solid to gas without passing through the liquid state. We can plot the sublimation quickly to confirm this:

In [ ]:
RESULTS.SUBLIMATION.plot();

## Next steps
[Back to overview](welcome.ipynb)

[Sensitivity studies with COSIPY](sensitivity_study.ipynb)